In [156]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.tree import plot_tree

from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

数据前处理

In [4]:
rawdat=pd.read_csv("./data/data.csv")

In [8]:
rawX=rawdat.loc[:,"XXXinorg1":"purity"] # 删除title与outcome

In [6]:
Y=np.array(rawdat.loc[:,"outcome"],dtype=np.float64)

In [13]:
ohe = OneHotEncoder()
class_le = LabelEncoder()

In [33]:
colLabel=list(traindat)# 列名的数组

In [98]:
datLabel=[]
X=np.array([],dtype=np.float64).reshape(len(traindat),-1)
errorCol=[] # 转换出错的文件
for i in range(292):
    try:
        col=np.array(traindat.iloc[:,i],dtype=np.float64).reshape(-1,1)
    except:
        if(len(set(traindat.iloc[:,i]))<=2):
            col=np.array(traindat.iloc[:,i]=='no',dtype=np.float64).reshape(-1,1)
        else:
            print(i)
            col=ohe.fit_transform(np.array(traindat.iloc[:,1]).reshape(-1,1)).toarray()
            if(col.shape[1]>100):
                errorCol.append(colLabel[i])
                continue
    datLabel+=[colLabel[i]]*(col.shape[1])
    processedDat=np.concatenate((processedDat,np.array(col).reshape(-1,1)),axis=1)

0
3
6
9
12
20
22


In [101]:
np.save("./data/processedDat.npy",processedDat)
np.save("./data/processedDatLabel.npy",datLabel)
np.save("./data/processedErrorFeature.npy",errorCol)

SVM

In [134]:
def crossValidation(X,y,n_splits,Model,params):
    print(params)
    for train_index, test_index in kf.split(traindat):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model=Model(**params)
        model.fit(X_train,y_train)
        pred=model.predict(X_test)
        print("recall={:.3f}".format(recall_score(y_test,pred,average='weighted')))
        print("precision={:.3f}".format(precision_score(y_test,pred,average="weighted")))
        print("confusion matrix is")
        print(confusion_matrix(y_test,pred))

In [149]:
classifiers = [
    (SVC,{"gamma":2, "C":1}),
    (GaussianProcessClassifier,{"kernel":1.0 * RBF(1.0)}),
    (DecisionTreeClassifier,{"max_depth":5}),
    (RandomForestClassifier,{"max_depth":5, "n_estimators":10, "max_features":1}),
    (MLPClassifier,{"alpha":1, "max_iter":1000}),
    (AdaBoostClassifier,{}),
    (GaussianNB,{}),
    (QuadraticDiscriminantAnalysis,{})]

In [151]:
for i in classifiers:
    crossValidation(processedDat,label,1,i[0],i[1])

{'gamma': 2, 'C': 1}
recall=0.374
precision=0.436
confusion matrix is
[[59  0 90  0]
 [ 0  0 59  0]
 [ 0  0 89  0]
 [ 0  0 99  0]]
recall=0.452
precision=0.413
confusion matrix is
[[ 36   0  22   2]
 [  0   0  88   5]
 [  0   0 136   1]
 [  0   0  99   7]]
recall=0.263
precision=0.590
confusion matrix is
[[  1   0 164   0]
 [  0   0  58   0]
 [  0   0 100   2]
 [  0   0  68   3]]
recall=0.480
precision=0.541
confusion matrix is
[[  0   0  34   0]
 [  0   0  35   0]
 [  0   0 182   1]
 [  0   0 136   8]]
recall=0.412
precision=0.599
confusion matrix is
[[  0   0  35   1]
 [  0   0  13   0]
 [  1   0 145   1]
 [  0   3 179  18]]
recall=0.585
precision=0.640
confusion matrix is
[[  0   0  30   0]
 [  0   0   4   0]
 [  0   0 199   4]
 [  0   0 126  32]]
recall=0.519
precision=0.546
confusion matrix is
[[  1   0  14   0]
 [  0   3  43   2]
 [  0   2 176  21]
 [  0   0 108  25]]
recall=0.327
precision=0.430
confusion matrix is
[[  3   0  24   0]
 [  0   0  26   0]
 [  0   6 108  19]
 [  0  

KeyboardInterrupt: 

将模型解释为决策树, 使用sklearn.tree.plot_tree进行可视化

In [ ]:
def reinterpret(X,model):
    pred=model.predict(X)
    ret=DecisionTreeClassifier()
    ret.fit(X,pred)
    return ret
    

In [154]:
tree=DecisionTreeClassifier()